# Моделирование нагрузки велокурьеров (на примере склада №5018)

Велозона разбита на несколько сегментов: Кушелевская, Лесная, Блюхер, Кондратьевский-верх, Кондратьевский-низ, Революция (часовая зона)

Формальная очередь одна для всех зон

Неформальных очередей 6:
Кушелевская+Блюхер(опционально),
Лесная+Блюхер(опционально),
Кондратьевский-верх+Блюхер(опционально),
Кондратьевский-низ зона 30 мин,
Кондратьевский-низ часовая зона + Революция (часовая зона),
Экспрессы (все зоны)

Курьеры распределены по четырем группам:
Группа "Экспресс" (2-5 чел)
Группа "Ревал" (1-3 чел)
Группа "30-минут" (2-4 чел)
Группа "Общая" (5-25 чел)

Координация между группами отсутствует, в случае "подгорания" заказов, куртор или тот кто его замещает, говорит (т.е. это ручная работа) кому и куда ехать (в этом случае курьер берется из любой группы, как правило из общей).

Курьер смотрит через мобильное приложение "ВкусВилл курьер" на формальную очередь, в которой отображаются все заказы.
Одновременно он смотрит на список каналов в телеграм, где отображаются "неформальные" очереди. Курьер встает в очередь, ставя плюс в неформальную очередь.
После завершения набора, он ставит себе 👍 (лайк)

Группа "Общая" обслуживает одновременно три "неформальных" очереди: Кушелевская+Блюхер(опционально), Лесная+Блюхер(опционально), Кондратьевский-верх+Блюхер(опционально)
Группа "Экспресс" обслуживает одну "неформальную" очередь: Экспрессы
Группа "Ревал" обслуживает одну "неформальную" очередь: Кондратьевский-низ часовая зона + Революция (часовая зона)
Группа "30-минут" обслуживает одну "неформальную" очередь: Кондратьевский-низ зона 30 мин

Некоторое время назад была введена "блокировка" формальной очереди, речи идет о галочке, когда курьер должен взять первый заказ из формальной очереди для начала формирования маршрута. Ниже, исследование влияния этой блокировки на общую пропускную способность потока заказов.

Файл с нагрузкой, сгенерирован путем равномерного размазывания заказов на неделю. Используются рандомные значения, за неимением доступа к реальным данным.

Модель поведения курьера: Курьер набирается по максимуму что бы отвезти как можно больше заказов и не сделать "просрок", подобная модель поведения приводит к прижатию к верхней границе SLA для заказа.


## Инициализация фабрики заказов (геокодинг адресов)

In [3]:
from os import environ
from os.path import exists

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from httpx import get

_ = load_dotenv(find_dotenv())

order_addresses = pd.read_csv('./order-addresses.csv')

# LOAD GEOCODING-CACHE
YANDEX_JAVASCRIPT_API_KEY = environ.get('YANDEX_JAVASCRIPT_API_KEY', '')
if not YANDEX_JAVASCRIPT_API_KEY:
    raise Exception('Invalid yandex javascript API key')

# REFRESH-GEOCODING-CACHE
GEOCODING_CACHE_FILENAME = './.geocoding-cache.csv'
_geocoding_cache_csv = pd.read_csv(GEOCODING_CACHE_FILENAME) if exists(GEOCODING_CACHE_FILENAME) else pd.DataFrame(columns=['geocoding_cache_address','geocoding_cache_lat','geocoding_cache_lon'])

# DEBUG
_geocoding_cache_csv.head()
_geocoding_cache_csv.tail()

for order_address in order_addresses.itertuples():
    geocoding_cache_address = order_address.order_address
    value = _geocoding_cache_csv['geocoding_cache_address'].loc[lambda x: x == geocoding_cache_address].index.any()
    if not value:
        # geocoding_cache_address','geocoding_cache_lat','geocoding_cache_lon
        response = get(f'https://geocode-maps.yandex.ru/1.x/?apikey={YANDEX_JAVASCRIPT_API_KEY}&geocode={geocoding_cache_address}&format=json')
        if response.status_code == 200:
            response_data = response.json()['response']['GeoObjectCollection']['featureMember']
            latitude, longitude = tuple(response_data[0]['GeoObject']['Point']['pos'].split(' ')) if response_data else tuple(0.0, 0.0)
            row = [geocoding_cache_address, latitude, longitude]
            _geocoding_cache_csv = pd.concat([pd.DataFrame([row], columns=_geocoding_cache_csv.columns), _geocoding_cache_csv], ignore_index=True)

try:
    _geocoding_cache_csv.to_csv(GEOCODING_CACHE_FILENAME, mode='x', index=False)
except FileExistsError:
    _geocoding_cache_csv.to_csv(GEOCODING_CACHE_FILENAME, index=False)

# orders_generator.tail()
orders_generator = order_addresses.\
    merge(_geocoding_cache_csv, right_on=['geocoding_cache_address'], left_on=['order_address']).\
        drop(columns=['geocoding_cache_address'])

orders_generator.head()

,order_address_id,order_address,order_address_capacity_per_week,order_address_average_bill,geocoding_cache_lat,geocoding_cache_lon
0,1,Новолитовская 12,50,1000,30.357990,59.982356
1,2,Новолитовская 14,140,1000,30.358673,59.983707
2,3,Новолитовская 10,160,1000,30.355268,59.981793
3,4,Грибалевой 7к1,160,1000,30.354020,59.983283
4,5,Грибалевой 7к3,80,1000,30.356714,59.983675


## методика вычисления среднего времени доставки и риска "просрока"

Среднее время доставки, интегральный показатель, который вычисляется по всем доставленным заказам.

$D = \frac{\sum_{n=1}^{N} t_n}{N}$ где $N$ общее количество заказов за период, $t_n$ время доставки.

Риск просрока вычисляется исходя из близости времени доставки к SLA.

$R = \frac{\sum_{n=1}^{N} (1 - \frac{T_n - t_n}{T_n})}{N}$ где $N$ общее количество заказов за период, $T_n$ SLA зоны доставки, $t_n$ время доставки.